In [1]:
import json

with open('transilieration-to-arabic.json') as json_file:
    computerLatinToArabicText = json.load(json_file)

with open('signs_mapper.json') as json_file:
    signs_mapper = json.load(json_file)

with open('dictionary.json') as json_file:
    dictionary = json.load(json_file)


In [5]:
def fromTransliterationToArabicAlphabet(word):
  arabic_word = ""
  if(not word):
      return ""
#   try:
#       arabic_word += computerLatinToArabicText[word[0]]
#   except:
#       print(word)
            
  for letter_index in range(0, len(word)):
        if(word[letter_index] == '.'):
            continue
        arabic_word += computerLatinToArabicText[word[letter_index]]
  return arabic_word

In [3]:
words= []
missing = []
for word in dictionary:
    arabic_word = fromTransliterationToArabicAlphabet(word['Egyptian'])
    try:
        sign = signs_mapper[word['Symbol'][-1]]
    except:
        missing.append(word['Symbol'][-1])
    word['ArabicAlphabet'] = arabic_word    
    word['Sign'] = sign
    words.append(word)
    # print(word['Translation'], sign, arabic_word)
    

In [4]:
len(words)

11770

In [ ]:
from deep_translator import GoogleTranslator

# Iterate over words in batches
for word_index in range(0, len(words)):
  # Translate batch
  translated_batch = GoogleTranslator(
    source='en', target='ar'
    ).translate_batch(words[word_index]['Translation'])
  words[word_index]['Arabic'] = translated_batch
  print(word_index)
  

In [99]:
words[5813]

{'Egyptian': 'mr',
 'Symbol': ['N36', 'N21', 'Z1'],
 'Translation': ['canal', 'artificial lake'],
 'ArabicAlphabet': 'مر',
 'Sign': '𓏤',
 'Arabic': ['قناة', 'بحيرة اصطناعية']}

In [5]:
words=[]

In [6]:
with open("words-final--.json", "w") as outfile: 
    json.dump(words, outfile, indent = 4, ensure_ascii=False, sort_keys=True) 

In [7]:
len(words)

0

In [13]:
# {
#     "Arabic": [{"Word": "هرم"}],
#     "Egyptian": [{
#         "Word": "مر",
#         "Symbol": "𓉴",
#         "Transliteration": "mr",
#         "Hieroglyphics": ["O24"]
#         }
#     ]
# }
def wordToRavenDBInput(word):
    return {
    "English": [{"Word": english_word} for english_word in word['Translation']],
    "Arabic": [{"Word": arabic_word} for arabic_word in word['Arabic']],
    "Egyptian": [{
        "Word": word['ArabicAlphabet'],
        "Symbol": word['Sign'],
        "Transliteration":  word['Egyptian'],
        "Hieroglyphics": word['Symbol']
        }
    ]
    }

In [14]:
wordToRavenDBInput(words[11])

{'English': [{'Word': '(n.) cargo'}],
 'Arabic': [{'Word': '(ن.) حمولة'}],
 'Egyptian': [{'Word': 'اتپو',
   'Symbol': '𓏛',
   'Transliteration': 'Atpw',
   'Hieroglyphics': ['A9', 'Q3', 'Z7', 'Y1', 'Z2']}]}

In [15]:
import requests
import time
# Define the batch size
batch_size = 200

# Iterate over words in batches
for i in range(0, len(words), batch_size):
  # Get the current batch
  batch = words[i:i+batch_size]
  batch_input = []
  for word in batch:
    batch_input.append(wordToRavenDBInput(word))
  # Post the batch to the website
  response = requests.post('https://bastet-server-ef94bb4e91eb.herokuapp.com/bulk', json={"words": batch_input})
  time.sleep(5)
  # Check the response status code
  if response.status_code == 201:
    print(f"Batch {i//batch_size+1} posted successfully")
  else:
    print(f"Failed to post batch {i//batch_size+1}", response.text)
  


Batch 1 posted successfully
Batch 2 posted successfully
Batch 3 posted successfully
Batch 4 posted successfully
Batch 5 posted successfully
Batch 6 posted successfully
Batch 7 posted successfully
Batch 8 posted successfully
Batch 9 posted successfully
Batch 10 posted successfully
Batch 11 posted successfully
Batch 12 posted successfully
Batch 13 posted successfully
Batch 14 posted successfully
Batch 15 posted successfully
Batch 16 posted successfully
Batch 17 posted successfully
Batch 18 posted successfully
Batch 19 posted successfully
Batch 20 posted successfully
Batch 21 posted successfully
Batch 22 posted successfully
Batch 23 posted successfully
Batch 24 posted successfully
Batch 25 posted successfully
Batch 26 posted successfully
Batch 27 posted successfully
Batch 28 posted successfully
Batch 29 posted successfully
Batch 30 posted successfully
Batch 31 posted successfully
Batch 32 posted successfully
Batch 33 posted successfully
Batch 34 posted successfully
Batch 35 posted success

In [8]:
with open('words-final--.json') as json_file:
    words = json.load(json_file)

In [9]:
len(words)

8523

In [6]:

for word in words: 
  word['ArabicAlphabet'] = fromTransliterationToArabicAlphabet(word['Egyptian'])
  

In [7]:

for word in words: 
  symbols = [ symbol for symbol in word['Symbol'] if symbol]
  if(symbols[-1] in ['Z1', 'Z2', 'Z3', ] and len(symbols) >=1):
    try:
      word['Sign'] = signs_mapper[symbols[-2]]
    except:
      try:
        word['Sign'] = signs_mapper[symbols[-2][:-1]]
      except:
        print(word)


{'Arabic': ['بدوي'], 'ArabicAlphabet': 'منثو', 'Egyptian': 'mnTw', 'Sign': '𓏪', 'Symbol': ['Y5', 'X1', 'G43', 'C248', 'Z3'], 'Translation': ['Bedouin']}


In [10]:
with open("words-final--.json", "w") as outfile: 
    json.dump(words, outfile, indent = 4, ensure_ascii=False, sort_keys=True) 

In [8]:
unique_words = []
seen = set()

for word in words:
  identifier = (word['Sign'], word['Egyptian'])
  if identifier not in seen:
    unique_words.append(word)
    seen.add(identifier)

words = unique_words

In [11]:
words[:10]

[{'Arabic': ['(فعل مصدري) يرفع على عالي', 'رفعت'],
  'ArabicAlphabet': 'قاإ',
  'Egyptian': 'qAi',
  'Sign': '𓏏',
  'Symbol': ['A28', 'X1'],
  'Translation': ['(v. infinitive) be raised on high', 'uplifted']},
 {'Arabic': ['(لاحقة prn.) أنا', 'أنا', 'لي'],
  'ArabicAlphabet': 'إ',
  'Egyptian': 'i',
  'Sign': '𓀀',
  'Symbol': ['A1'],
  'Translation': ['(suffix prn.) I', 'me', 'my']},
 {'Arabic': ['(اسم جماعي) رجل', 'الرجال', 'الجنس البشري', 'المصريون'],
  'ArabicAlphabet': 'رمث',
  'Egyptian': 'rmT',
  'Sign': '𓀀',
  'Symbol': ['A1', 'A1', 'A1'],
  'Translation': ['(collective n.) man', 'men', 'mankind', 'Egyptians']},
 {'Arabic': ['شرب'],
  'ArabicAlphabet': 'سور',
  'Egyptian': 'swr',
  'Sign': '𓀁',
  'Symbol': ['A2'],
  'Translation': ['drink']},
 {'Arabic': ['نقي', 'تطهير الذات', 'الإستحمام', 'تطهير', 'طهارة', 'نقاء'],
  'ArabicAlphabet': 'وعب',
  'Egyptian': 'wab',
  'Sign': '𓀆',
  'Symbol': ['A6'],
  'Translation': ['pure ',
   'puri fy one self',
   'bathe',
   'cleanse',
   'pu